In [1]:
import importlib
import src.chat_bot
importlib.reload(src.chat_bot)
from src.chat_bot import ExconManual

In [2]:
#ad = True
ad = False # = ADLA chat
if ad:
    print("Loaded Authorised Dealer Manual")
else:
    print("Loaded ADLA manual")

logging_level = 15
#logging_level = 20
log_file = ''

if ad:
    path_to_manual_as_csv_file = "./inputs/ad_manual.csv"
    path_to_definitions_as_parquet_file = "./inputs/ad_definitions.parquet"
    path_to_index_as_parquet_file = "./inputs/ad_index.parquet"
    chat_for_ad = True
else:
    path_to_manual_as_csv_file = "./inputs/adla_manual.csv"
    path_to_definitions_as_parquet_file = "./inputs/adla_definitions.parquet"
    path_to_index_as_parquet_file = "./inputs/adla_index.parquet"
    chat_for_ad = False


excon = ExconManual(path_to_manual_as_csv_file, path_to_definitions_as_parquet_file, path_to_index_as_parquet_file, chat_for_ad = chat_for_ad, log_file=log_file, logging_level=logging_level)


Loaded ADLA manual


In [4]:
print(excon.get_regulation_detail("A.3 (B)(ii)(a)"))

A.3 The terms and conditions for submission of an application for authorisation to conduct the business of an Authorised Dealer in foreign exchange with limited authority
    (B) Application procedure
        (ii) The completed prescribed 'Application for authorisation to conduct the business of an Authorised Dealer in foreign exchange with limited authority, a copy of which is attached hereto as Annexure A, must be accompanied by the following information:
            (a) a certified copy of the Notice of Incorporation (Form CoR14.1) issued by the CIPC, as proof of the registration of the limited liability company in South Africa under the Companies Act, 2008 (Act No. 71 of 2008);
        (ii) the manner in which and the processes by which the ADLA conducts additional CDD measures in respect of legal persons, trusts and partnerships;


In [33]:
model_to_use="gpt-3.5-turbo"
#model_to_use = "gpt-4-1106-preview"
#model_to_use="gpt-4"
#model_to_use="gpt-3.5-turbo-16k"

In [34]:
#user_context = "A non-resident wants to buy residential property in South Africa. What are the restrictions around doing this?"
#user_context = "A South African citizen is emigrating. What is the process they need to follow to take all their cash offshore?"
#user_context = "What is the Foreign investment limits for South African institutional investors?"
#user_context = "Can a individual transfer his shares in a dual listed company to his offshore portfolio?"
#user_context = "Can cash balances for individuals who are no longer tax resident be transferred offshore?"
#user_context = "How do I need to report outbound cross-boarder payments?"
#user_context = "Are there different categories for ALDAs?"
#user_context = "Who can trade gold?"

#user_context = "What documentary evidence is required?"
#user_context = "What does cemad say about travel services?"
#user_context = "Can I import gold?"
#user_context = "how much money can an individual take offshore?"
#user_context = "How much can an individual take offshore in a year?"
#user_context = "what is the limit for commodities"
user_context = "what does the regulation state about travel?"
#user_context = "How much can adult residents use for travel?"

user_context = "If I want to offer cross-border payments, what license do I need?"

excon.reset_conversation_history()
excon.user_provides_input(user_context, threshold=0.15, model_to_use=model_to_use, temperature=0, max_tokens=300)
print(excon.messages[-1]["content"])


INFO:src.chat_bot:User Question:
If I want to offer cross-border payments, what license do I need?
DEV:src.chat_bot:#################   Similarity Search       #################
DEV:src.chat_bot:--   No relevant definitions found
DEV:src.chat_bot:--   Relevant Sections
DEV:src.chat_bot:0.1282:               C.1(E):        question: What are the reporting requirements for cross-border transactions?
DEV:src.chat_bot:0.1494:         B.2(A)(xxii):        question: What are the requirements regarding foreign currency?
DEV:src.chat_bot:0.1495:          B.1(B)(iii):        question: What guidelines should I follow while offering money remittance services?
DEV:src.chat_bot:0.1498:            B.2(A)(x):        question: Are there specific rules for launching digital foreign exchange services?
DEV:src.chat_bot:Top result: C.1(E) with a cosine distance of 0.1282
DEV:src.chat_bot:No unique mode
DEV:src.chat_bot:Only the top result added but more were found. Adding the next most likely answer
DEV:s

INFO:src.chat_bot:assistant:
 If you want to offer cross-border payments, you would need to apply to the Financial Surveillance Department to become a Category Two ADLA. Category Two ADLAs are Bureaux de Change who can also offer money remittance services in partnership with external money transfer operators to facilitate cross-border transfers. However, there are certain principles that must be complied with, such as taking responsibility for compliance with the requirements of the FIC Act, having adequate resources and systems in place, guaranteeing that all information will be held and maintained in South Africa, and having adequate client interface procedures and capturing systems in place. (CEMADLA B.1(B)(iii)(a)-(dd))


If you want to offer cross-border payments, you would need to apply to the Financial Surveillance Department to become a Category Two ADLA. Category Two ADLAs are Bureaux de Change who can also offer money remittance services in partnership with external money transfer operators to facilitate cross-border transfers. However, there are certain principles that must be complied with, such as taking responsibility for compliance with the requirements of the FIC Act, having adequate resources and systems in place, guaranteeing that all information will be held and maintained in South Africa, and having adequate client interface procedures and capturing systems in place. (CEMADLA B.1(B)(iii)(a)-(dd))


In [7]:
excon.messages

[{'role': 'user',
  'content': "Question: what does the regulation state about travel?\n\nSections from the Manual\nB.4 Single discretionary allowance and other miscellaneous payments for private individuals\n    (B) Travel allowances\n        (i) General\n            (a) There is no limit on the amount of Rand that may be spent on travelling within the CMA.\n            (b) Residents (natural persons) who are under the age of 18 years may not be accorded single discretionary allowance as outlined in subsection (A)(i) above, but may be accorded a travel allowance not exceeding an amount of R200 000 per calendar year.\n            (c) In addition, up to R25 000 in cash, per person, may be taken when proceeding on visits outside the CMA, to meet travellers' immediate needs on return to South Africa.\n            (d) When according travel allowances to persons who do not conduct accounts in their books, Authorised Dealers must record such persons' names, nationalities, residential address

In [60]:
user_context = "hi"
#user_context = "but what category do I need to be?"
excon.user_provides_input(user_context, threshold=0.15, model_to_use=model_to_use, temperature=0, max_tokens=300)
print(excon.messages[-1]["content"])

INFO:src.chat_bot:User Question:
hi
DEV:src.chat_bot:#################   Similarity Search       #################
DEV:src.chat_bot:--   No relevant definitions found
DEV:src.chat_bot:--   No relevant sections found
DEV:src.chat_bot:#################   RAG Prompts   #################
INFO:src.chat_bot:System Prompt:
You are answering questions for an Authorised Dealer with Limited Authority (ADLA) based only on the relevant sections from the 'Currency and Exchange Manual for Authorised Dealers in foreign exchange with limited authority' (Manual or CEMADLA) that are provided. You have three options:
1) Answer the question. Preface an answer with the tag 'ANSWER:'. If possible, end the answer with the reference to the section or sections you used to answer the question.
2) Request additional documentation. If, in the body of the sections provided, there is a reference to another section of the Manual that is directly relevant and not already provided, respond with the word 'SECTION:' fol

To offer cross-border payments, you would need to be a Category Two ADLA. (CEMADLA B.1(B)(iii)(a))


In [8]:
import importlib
import src.valid_index
importlib.reload(src.valid_index)
from src.valid_index import get_excon_manual_index
import re

index = get_excon_manual_index()


In [5]:
s = " B.4(B)(ii)(a) of the Manual states that foreign currency may be made available within the single discretionary allowance limit of R1 million per calendar year to eligible parties. However, the specific limits for adults are not mentioned in the provided sections. Please refer to the relevant section(s) for more information on the limits for travel allowances for adults."
#s = "B.4(B)(ii)(a)"

print(index.extract_valid_reference(s))


None


In [22]:
patterns = [
        r'[A-Z]\.\d{0,2}',
        r'\([A-Z]\)',
        r'\(\b(i|ii|iii|iv|v|vi|vii|viii|ix|x|xi|xii|xiii|xiv|xv|xvi|xvii|xviii|xix|xx|xxi|xxii|xxiii|xxiv|xxv|xxvi|xxvii)\b\)',
        r'\([a-z]\)',
        r'\([a-z]{2}\)',
        r'\((?:[1-9]|[1-9][0-9])\)'
    ]
matches = [re.findall(pattern, s) for pattern in patterns]
matches

[['B.4'], ['(B)'], ['ii'], ['(a)', '(s)'], ['(ii)'], []]

In [26]:
match = re.search(r'\([a-z]\)', s)
match

<re.Match object; span=(11, 14), match='(a)'>

In [30]:
s[match.end()]

' '

In [87]:
excon.get_regulation_detail("B.5(B)(iii)")

'B.5 Personal transfers by foreign nationals and immigrants\n    (B) Immigrants\n        (iii) Payments by immigrants\n            (a) Applications by immigrants may be approved by Authorised Dealers for the provision of foreign currency to cover current and arrear premiums due on foreign currency life insurance policies or contributions to pension and medical aid funds, provided that:\n                (aa) the necessary declaration and undertaking were completed; and\n                (bb) documentary evidence is presented, verifying the amounts due and that the commitment was entered into before the applicants took up residence in South Africa.\n            (b) Immigrants may against the presentation to an Authorised Dealer of documentary evidence confirming the amount involved repay loans received in their previous country of domicile.\n            (c) Immigrants may settle foreign tax commitments from South Africa against the production of documentary evidence confirming the amount 

In [11]:
print(excon._create_system_message())

You are answering questions for an Authorised Dealer (AD) based only on the relevant sections from the 'Currency and Exchange Manual for Authorised Dealers' (Manual or CEMAD) that are provided. You have three options:
1) Answer the question. Preface an answer with the tag 'ANSWER:'. If possible, end the answer with the reference to the section or sections you used to answer the question.
2) Request additional documentation. If, in the body of the sections provided, there is a reference to another section of the Manual that is directly relevant and not already provided, respond with the word 'SECTION:' followed by the section reference.
3) State 'NONE:' and nothing else in all other cases

Note: In the manual sections are numbered like A.1(A) or C.(C)(iii)(c)(cc)(3). The first index uses the regex pattern r'[A-Z]\.\d(0, 2)'. Thereafter, each sub-index is surrounded by round brackets


In [23]:
import pandas as pd
df_text_all = pd.read_parquet(path_to_index_as_parquet_file, engine='pyarrow')
df_text_all

,section,text,Embedding,source
0,Legal context,Legal context,"[0.008710513822734356, -0.01103331707417965, 0...",heading
1,A.2(A),Authorised entities. Authorised Dealers,"[-0.009998462162911892, -0.009398686699569225,...",heading
2,A.2(B),Authorised entities. Authorised Dealers in for...,"[-0.009065092541277409, -0.012516901828348637,...",heading
3,A.3(A),Duties and responsibilities of Authorised Deal...,"[0.009497863240540028, -0.01100919209420681, -...",heading
4,A.3(B),Duties and responsibilities of Authorised Deal...,"[-0.010621572844684124, -0.014073583297431469,...",heading
...,...,...,...,...
1493,J.(G),What is the procedure for reporting outward pa...,"[-0.0016078203916549683, -0.000447303900728002...",question
1494,J.(G),Rules for reporting outward payments using the...,"[0.005922725424170494, -0.004037913400679827, ...",summary
1495,B.4(G)(i),How much money can an individual take offshore?,"[0.01233571209013462, -0.03362925723195076, 0....",manual
1496,B.4(B)(i),Is there any general guidance on travel?,"[0.0002241560723632574, 0.000385825231205672, ...",manual


In [ ]:
excon.get_regulation_detail('I.(1)(E)(ii)')

'No section could be found with the reference I.(E)(ii)'

In [ ]:
#user_context = "What is a CFC Account?"
#user_context = "I am submitting documentation to the financial surveillance department. What information do I need to include in my submission?" # (A.3 (B))
#user_context = "Can a bank provide foreign currency denominated facilities to a South African Corporate?" #  (A.3 (B))
#user_context = "For how long does the approval for a facility remain valid?" #  (A.3 (B))
#user_context = "Other than authorised transaction requests, are there requirements for authorised dealers to submit information to the financial surveillance department?" #  (A.3 (B))
#user_context = "I have been asked to act on an approval issued to another bank. Can I do that?" #  (A.3 (B))
#####user_context = "--- Same as above but use "Authorised Dealer" instead of "Bank"
#user_context = "If I am concerned that as client may in breach of regulations, what should I do?" #  (A.3 (B))
#user_context = "What constitutes 'documentary evidence' for a foreign currency transaction?" #  (A.3 (C))
#user_context = "Is Swaziland part of the CMA?" #  (A.3 (E))
#user_context = "A client wants to buy Namibian Dollars. What approvals do they need before they can do this?" #  (A.3 (E))
#user_context = "A person from Lesotho wants to buy dollars from me. Can I sell them the dollars?" #  (A.3 (E))
#user_context = "A person from Namibia wants to buy rand from me. Can I sell them the rand?" #  (A.3 (E))
#user_context = "A treasury management company wants to trade dollars with me. What do I need to check before I can do this?" #  (A.4 (B))
#user_context = "A client wants to import goods and pay in foreign currency. What documentation do I need before I can assist with the transaction" #  (B.1 (B))
#user_context = "Over and above the price of the goods, what additional costs can be covered in foreign currency?" #  (B.1 (C))
#user_context = "Can advance payment be made in a foreign currency to cover imported goods?" #  (B.1 (D)(i))

#user_context = "Can advance payment be made in a foreign currency to cover imported capital goods?" #  (B.1 (D)(ii))
#user_context = "When paying for imports in cash, what documentation is required before providing the foreign currency?" #  (B.1 (D)(iii))

#user_context = "Can imported goods be financed over more than 12 months?" #  (B.1 (D)(iv))
#user_context = "How much money can an individual invest internationally?" #  ?
#user_context = "How much money can an individual take offshore?" #  ?

#user_context = "A media company want to list offshore. Are there specific conditions for this?"
# user_context = "How many rands can a resident take offshore in cash?"
#user_context = "Can I grant a foreign company a Rand facility to pay for goods they wish to purchase from a South African exporter?"
#user_context = "Can I support a non-residential entity with short-term rand trade finance for their exports from South Africa?"
#user_context = "Hi"
#user_context = "Who can trade gold?"
#user_context = "are there specific exchange control rules for religious organizations?"
#user_context = "How much foreign currency can a charity take offshore each year?"

